In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from torch.optim import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset

In [2]:
print(f'PyTorch version: {torch.__version__}')
print('*'*10)
print(f'_CUDA version: ')
!nvcc --version
print('*'*10)
print(f'CUDNN version: {torch.backends.cudnn.version()}')
print(f'Available GPU devices: {torch.cuda.device_count()}')
print(f'Device Name: {torch.cuda.get_device_name()}')

PyTorch version: 2.5.1+cu118
**********
_CUDA version: 
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Wed_Oct_30_01:18:48_Pacific_Daylight_Time_2024
Cuda compilation tools, release 12.6, V12.6.85
Build cuda_12.6.r12.6/compiler.35059454_0
**********
CUDNN version: 90100
Available GPU devices: 1
Device Name: NVIDIA GeForce RTX 4070 Ti SUPER


## Zdobycie danych

In [10]:
DATA_PATH = os.path.abspath(os.path.join('..', '..', 'data', 'processed', 'ravdess'))

In [11]:
def load_data(data_path, use_distinguished_points=False):
    all_data = []
    all_labels = []

    for file in os.listdir(data_path):
        if file.endswith(".npy"):
            data = np.load(os.path.join(data_path, file), allow_pickle=True)
            data = np.array(data, dtype=np.float32)

            all_data.append(data)

            label = int(file.split("-")[2])
            all_labels.append(label)

    return np.array(all_data, dtype=object), np.array(all_labels)

In [12]:
all_data, all_labels = load_data(DATA_PATH)

## Preprocessing danych

In [13]:
def preprocess_data(data, labels):
    tensor_data = [torch.tensor(d, dtype=torch.float32) for d in data]
    padded_data = pad_sequence(tensor_data, batch_first=True)

    encoder = LabelBinarizer()
    encoded_labels = encoder.fit_transform(labels)
    encoded_labels = torch.tensor(encoded_labels, dtype=torch.float32)

    X_train, X_temp, y_train, y_temp = train_test_split(
        padded_data, encoded_labels, test_size=0.3, random_state=42
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, random_state=42
    )

    return X_train, X_val, X_test, y_train, y_val, y_test

In [14]:
X_train, X_val, X_test, y_train, y_val, y_test = preprocess_data(all_data, all_labels)

In [15]:
print(X_train.shape)
print(y_train.shape)

torch.Size([2012, 157, 478, 2])
torch.Size([2012, 8])


In [16]:
# Class distribution
from collections import Counter

for key, val in sorted(Counter((label.argmax().item() for label in y_train)).items(), key=lambda i: i[0]):
    print(f"{key}:{val}")

0:139
1:271
2:258
3:266
4:262
5:278
6:273
7:265


# MODEL TORCH

### W podejściu wykorzystane zostaną 2 modele - pierwszy z nich będzie siecią konwolucyjną 2d, która będzie miała za zadanie nauczyć się rozpoznawać cechy charakterystyczne dla wybranej klatki (zbioru współrzędnych pkt charakterystycznych). Do klasyfikacji szeregu czasowego zostanie wykorzystana sekwencyjna sieć neuronowa LSTM.

## Zbudowanie modelu ekstrakcji cech

In [17]:
class EmotionClassifier(nn.Module):
    def __init__(self):
        super(EmotionClassifier, self).__init__()
        
        # Spatial feature extraction using Conv1D
        self.conv1 = nn.Conv1d(in_channels=2, out_channels=32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool1d(kernel_size=2)
        
        # LSTM layers for temporal feature extraction
        self.lstm1 = nn.LSTM(input_size=32 * 239, hidden_size=128, batch_first=True, bidirectional=True)
        self.lstm2 = nn.LSTM(input_size=128 * 2, hidden_size=64, batch_first=True)
        
        # Fully connected classification layer
        self.fc = nn.Linear(64, 8)  # 8 emotion classes

    def forward(self, x):
        # x shape: (batch_size, frames, landmarks, coordinates)
        batch_size, frames, landmarks, coordinates = x.shape
        
        # Reshape for Conv1D: (batch_size * frames, landmarks, coordinates)
        x = x.view(-1, landmarks, coordinates).permute(0, 2, 1)
        
        # Spatial feature extraction
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        
        # Flatten spatial features
        x = x.view(batch_size, frames, -1)  # (batch_size, frames, features)
        
        # Temporal feature extraction
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        
        # Classification
        x = self.fc(x[:, -1, :])  # Take the last timestep's output
        return x

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EmotionClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=1e-4)

## Trening modelu

In [19]:
BATCH_SIZE = 32
EPOCHS = 300

In [20]:
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [21]:
writer = SummaryWriter("runs/torch-lstm/emotion_classifier")

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        y_batch = y_batch.argmax(dim=1)
        
        # Forward pass
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(y_batch).sum().item()
        total += y_batch.size(0)
    
    train_acc = correct / total

    # Validation
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_batch = y_batch.argmax(dim=1)
            
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += predicted.eq(y_batch).sum().item()
            total += y_batch.size(0)
    
    val_acc = correct / total

    writer.add_scalar("Loss/Train", train_loss, epoch)
    writer.add_scalar("Loss/Validation", val_loss, epoch)
    writer.add_scalar("Accuracy/Train", train_acc, epoch)
    writer.add_scalar("Accuracy/Validation", val_acc, epoch)

    print(f"Epoch {epoch + 1}/{EPOCHS}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
    
writer.close()


Epoch 1/300, Train Loss: 130.4405, Train Acc: 0.1337, Val Loss: 28.8861, Val Acc: 0.1276
Epoch 2/300, Train Loss: 130.1196, Train Acc: 0.1307, Val Loss: 28.8689, Val Acc: 0.1276
Epoch 3/300, Train Loss: 130.0657, Train Acc: 0.1332, Val Loss: 28.8478, Val Acc: 0.1276
Epoch 4/300, Train Loss: 129.7761, Train Acc: 0.1446, Val Loss: 28.7143, Val Acc: 0.1671
Epoch 5/300, Train Loss: 128.8254, Train Acc: 0.1849, Val Loss: 28.4541, Val Acc: 0.1903
Epoch 6/300, Train Loss: 126.4131, Train Acc: 0.1968, Val Loss: 28.3278, Val Acc: 0.1810
Epoch 7/300, Train Loss: 125.3803, Train Acc: 0.2058, Val Loss: 27.5940, Val Acc: 0.1879
Epoch 8/300, Train Loss: 124.1444, Train Acc: 0.2142, Val Loss: 27.4197, Val Acc: 0.1926
Epoch 9/300, Train Loss: 123.3773, Train Acc: 0.2033, Val Loss: 27.5928, Val Acc: 0.1995
Epoch 10/300, Train Loss: 123.2779, Train Acc: 0.2048, Val Loss: 27.2950, Val Acc: 0.1903
Epoch 11/300, Train Loss: 122.9259, Train Acc: 0.2177, Val Loss: 27.3352, Val Acc: 0.2019
Epoch 12/300, Train

## Ewaluacja modelu

In [22]:
model.eval()

test_loss = 0
correct = 0
total = 0

with torch.no_grad():
    for X_batch, y_batch in DataLoader(TensorDataset(X_test, y_test), batch_size=BATCH_SIZE, shuffle=False):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        y_batch = y_batch.argmax(dim=1)
        
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        
        test_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(y_batch).sum().item()
        total += y_batch.size(0)

test_loss /= len(y_test)
test_acc = correct / total

print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

Test Loss: 0.0383, Test Accuracy: 0.5417


# MODEL SEGLEARN

In [49]:
from sklearn.ensemble import RandomForestClassifier
from seglearn.pipe import Pype
from seglearn.transform import FeatureRep, Segment
from pandas import DataFrame
from keras.layers import Dense, LSTM, Conv1D
from keras.models import Sequential
from scikeras.wrappers import KerasClassifier

### Przekształcenie danych na wektor płaski połączonych współrzędnych

In [13]:
X_train_np = X_train.numpy()
X_val_np = X_val.numpy()
X_test_np = X_test.numpy()
y_train_np = y_train.numpy()
y_val_np = y_val.numpy()
y_test_np = y_test.numpy()

In [14]:
def flatten_landmarks(data):
    n_samples, n_timesteps, n_landmarks, n_coords = data.shape
    return data.reshape(n_samples, n_timesteps, n_landmarks * n_coords)

X_train_flat = flatten_landmarks(X_train_np)
X_val_flat = flatten_landmarks(X_val_np)
X_test_flat = flatten_landmarks(X_test_np)

In [15]:
print(X_train_flat.shape, y_train_np.shape)

(2012, 157, 956) (2012, 8)


### Budowa modelu

In [ ]:
# def crnn_model(width=100, n_vars=956, n_classes=8, conv_kernel_size=5,
#                conv_filters=3, lstm_units=3):
#     input_shape = (width, n_vars)
#     model = Sequential()
#     model.add(Conv1D(filters=conv_filters, kernel_size=conv_kernel_size,
#                      padding='valid', activation='relu', input_shape=input_shape))
#     model.add(LSTM(units=lstm_units, dropout=0.1, recurrent_dropout=0.1))
#     model.add(Dense(n_classes, activation="softmax"))

#     model.compile(loss='categorical_crossentropy', optimizer='adam',
#                   metrics=['accuracy'])

#     return model

# pipe = Pype([('seg', Segment(width=20, step=10)),
#              ('crnn', KerasClassifier(build_fn=crnn_model, epochs=50, batch_size=8,
#                                       verbose=1, validation_split=0.2))])

In [61]:
from tensorflow.keras.callbacks import TensorBoard
import datetime

log_dir = "runs/seglearn/"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

pipe = Pype([
    ("segment", Segment(width=20, step=10)),  # Segmentacja sekwencji
    ("features", FeatureRep()),              # Ekstrakcja cech
    ("rf", RandomForestClassifier(n_estimators=100))  # RandomForest
])

### Trening modelu

In [62]:
pipe.fit(X_train_flat, y_train_np)

Pype(steps=[('segment', Segment(overlap=None, step=10, width=20)),
            ('features',
             FeatureRep(features={'abs_energy': <function abs_energy at 0x000001F571A24CC0>,
                                  'kurt': <function kurt at 0x000001F571A251C0>,
                                  'max': <function maximum at 0x000001F571A25080>,
                                  'mean': <function mean at 0x000001F571A24900>,
                                  'median': <function median at 0x000001F571A249A0>,
                                  'min': <function minimum at 0x000001F571A24FE0>,
                                  'mnx': <function mean_crossings at 0x000001F571A25440>,
                                  'mse': <function mse at 0x000001F571A253A0>,
                                  'skew': <function skew at 0x000001F571A25120>,
                                  'std': <function std at 0x000001F571A24D60>,
                                  'var': <function var at 0x000001F571A24E00>})),
            ('rf', RandomForestClassifier())])

In [63]:
history = pipe.history
print(DataFrame(history))

Empty DataFrame
Columns: []
Index: []


### Ewaluacja modelu

In [64]:
val_accuracy = pipe.score(X_val_flat, y_val_np)
test_accuracy = pipe.score(X_test_flat, y_test_np)

print(f"Dokładność na zbiorze walidacyjnym: {val_accuracy:.2f}")
print(f"Dokładność na zbiorze testowym: {test_accuracy:.2f}")

Dokładność na zbiorze walidacyjnym: 0.64
Dokładność na zbiorze testowym: 0.62


# TODYNET

### Przygotowanie danych

In [16]:
TodyNet_DATA_PATH = os.path.join("..", "src", "external", "TodyNet", "data", "UCR", "EMOTIONS")

os.makedirs(TodyNet_DATA_PATH, exist_ok=True)

In [17]:
X_train_tensor = torch.tensor(X_train_flat, dtype=torch.float32).unsqueeze(1)  # adding channel dimension
X_val_tensor = torch.tensor(X_val_flat, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test_flat, dtype=torch.float32).unsqueeze(1)

y_train_class = torch.argmax(y_train, dim=1)
y_val_class = torch.argmax(y_val, dim=1)
y_test_class = torch.argmax(y_test, dim=1)

# Save the data in PyTorch (.pt) format
torch.save(X_train_tensor, os.path.join(TodyNet_DATA_PATH, 'X_train.pt'))
torch.save(X_val_tensor, os.path.join(TodyNet_DATA_PATH, 'X_valid.pt'))
torch.save(X_test_tensor, os.path.join(TodyNet_DATA_PATH, 'X.pt'))

# Save the labels in PyTorch (.pt) format
torch.save(y_train_class, os.path.join(TodyNet_DATA_PATH, 'y_train.pt'))
torch.save(y_val_class, os.path.join(TodyNet_DATA_PATH, 'y_valid.pt'))
torch.save(y_test_class, os.path.join(TodyNet_DATA_PATH, 'y.pt'))

In [18]:
X_train_tensor.shape

torch.Size([2012, 1, 157, 956])

### Trening modelu [pool_ratio 0.8, ponieważ rozmiar danych jest zbyt duży na 0.2]

In [ ]:
# cd .\src\external\TodyNet\src\ & python train.py --dataset='EMOTIONS' --pool_ratio 0.8